# 수행목표
- 영화별 평균 평점으로 사용자별 평점과 비교한다.

# 수행단계
랜덤 평점은 뒤로 하고 이제 영화별로 평균 평점을 계산해서 비교한다.

- 추천 코드 class 명을 `AvgMovieRecommender로` 한다.
- 결과에 `rate_avg_movie` 필드를 추가해서 영화 평점 평균을 계산한 다음 업데이트 한다.
- `rate_avg_movie` 결과를 회귀 평가 한다.
- `rate_avg_movie_class` 필드를 추가해서 `rate_avg_movie` 값을 반올림한 값을 업데이트 한다.
- `rate_avg_movie_class` 결과를 분류 평가 한다.
- 결과 데이터 평점 높은 순 n개를 출력하고 그 아래에 분석 결과를 출력한다.
- 랜덤 추천과 평가를 비교한다.

# 라이브러리 설치

In [1]:
# Library

import os
import sys
import pandas as pd
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns

mpl.rcParams['font.family'] = 'AppleGothic'
mpl.rcParams['axes.unicode_minus'] = False

# DataLoader

In [2]:
class MovieDataLoader:
    def __init__(self, file_path):
        self.file_path = file_path
        movie_path = os.path.join(self.file_path, 'movies.txt')
        rate_path = os.path.join(self.file_path, 'rates.csv')
        self.movies = pd.read_csv(movie_path, sep='\t')
        self.rates = pd.read_csv(rate_path)

    def load(self):
        self._preprocess()

        return self.movies, self.rates

    def _preprocess(self):
        self.movies.dropna(subset=['title_eng'], inplace=True)

        if self.movies['year'].isnull().sum() > 0:
            non_year = self.movies[self.movies['year'].isnull()]
            for row in non_year.iterrows():
                movie = row[1]['movie']
                title_eng = row[1]['title_eng'].split(' , ')[:-1]
                title_eng = ' , '.join(title_eng)
                year = row[1]['title_eng'].split(' , ')[-1]
                self.movies.loc[self.movies['movie'] == movie, 'title_eng'] = title_eng
                self.movies.loc[self.movies['movie'] == movie, 'year'] = year

        if self.movies['grade'].isnull().sum() > 0:
            self.movies['grade'] = self.movies['grade'].fillna('NR grade')



In [3]:
movies_df, rates_df = MovieDataLoader('../data/kmrd/').load()

movies_df.info()
rates_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 991 entries, 0 to 998
Data columns (total 5 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   movie      991 non-null    int64 
 1   title      991 non-null    object
 2   title_eng  991 non-null    object
 3   year       991 non-null    object
 4   grade      991 non-null    object
dtypes: int64(1), object(4)
memory usage: 46.5+ KB
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 140710 entries, 0 to 140709
Data columns (total 4 columns):
 #   Column  Non-Null Count   Dtype
---  ------  --------------   -----
 0   user    140710 non-null  int64
 1   movie   140710 non-null  int64
 2   rate    140710 non-null  int64
 3   time    140710 non-null  int64
dtypes: int64(4)
memory usage: 4.3 MB


/var/folders/ph/w9kxghkd5959cwdbzq0m7_9m0000gn/T/ipykernel_52668/2801200881.py:25: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '1983' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  self.movies.loc[self.movies['movie'] == movie, 'year'] = year


# Random Recommend

In [4]:
# 결과에 random 평점 field(rate_random)를 추가하고 random 값을 반영해서 결과를 return 한다.
# 결과에 random 평점 field(rate_random)를 추가하고 random 값을 반영해서 결과를 return 한다.

class RandomRecommender:
    def __init__(self, movies_df, rates_df):
        self.movies_df = movies_df
        self.rates_df = rates_df

    def run(self, n_items=10):
        df = self.rates_df.copy()
        df['rate_random'] = np.random.rand(len(df)) * 10

        top_n = (
            df.groupby('user', group_keys=True)
            .apply(lambda x: x.nlargest(n_items, 'rate_random'))
        )

        top_n = top_n.reset_index(drop=True)
        top_n['rate_random_class'] = top_n['rate_random'].apply(lambda x: np.round(x))
        return top_n

    def run_for_user(self, user_id, n_items=10):
        user_rates = self.rates_df[self.rates_df['user'] == user_id].copy()
        rate_random = np.random.rand(len(user_rates)) * 10
        user_rates['rate_random'] = rate_random
        user_rates = user_rates.sort_values(by='rate_random', ascending=False)
        user_rates = user_rates.head(n_items)
        user_rates['rate_random_class'] = user_rates['rate_random'].apply(lambda x: np.round(x))


        return user_rates

In [5]:
result = RandomRecommender(movies_df, rates_df).run(20)
result

/var/folders/ph/w9kxghkd5959cwdbzq0m7_9m0000gn/T/ipykernel_52668/1294678380.py:14: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  df.groupby('user', group_keys=True)


,user,movie,rate,time,rate_random,rate_random_class
0,0,10262,10,1426916400,9.986923,10.0
1,0,10204,7,1411430520,9.962863,10.0
2,0,10023,7,1428738480,9.853472,10.0
3,0,10793,9,1423573320,9.851841,10.0
4,0,10215,7,1452407820,9.812458,10.0
...,...,...,...,...,...,...
117757,52023,10998,10,1421679660,0.472448,0.0
117758,52024,10998,10,1204622460,4.287392,4.0
117759,52025,10998,7,1498546920,3.518788,4.0
117760,52026,10998,9,1434090420,6.854743,7.0


# Analyzer

In [6]:
class Analyzer:
    def __init__(self, y_true, y_pred, y_pred_class):
        self.y_true = y_true
        self.y_pred = y_pred
        self.y_pred_class = y_pred_class

    def analyze_error(self):
        self._analyze_mae()
        self._analyze_mse()
        self._analyze_rmse()
        self._analyze_mape()

    def analyze_confusion_matrix(self):
        self._make_confusion_matrix()
        self._analyze_accuracy()
        self._analyze_precision()
        self._analyze_recall()
        self._analyze_f1()

    def _analyze_mae(self):
        self.mae = np.abs(self.y_true - self.y_pred).mean()
        print(f'MAE: {self.mae}')

    def _analyze_mse(self):
        self.mse = ((self.y_true - self.y_pred) ** 2).mean()
        print(f'MSE: {self.mse}')

    def _analyze_rmse(self):
        self.rmse = np.sqrt(self.mse)
        print(f'RMSE: {self.rmse}')

    def _analyze_mape(self):
        self.mape = (np.abs(self.y_true - self.y_pred) / self.y_true).mean()
        print(f'MAPE: {self.mape}')

    def _make_confusion_matrix(self):
        unique_labels = np.unique(np.concatenate((self.y_true, self.y_pred_class)))
        self.confusion_matrix = np.zeros((len(unique_labels), len(unique_labels)), dtype=int)

        label_to_index = {label: index for index, label in enumerate(unique_labels)}
        for true, pred in zip(self.y_true, self.y_pred_class):
            self.confusion_matrix[label_to_index[true]][label_to_index[pred]] += 1

        print('Confusion Matrix :')
        print(self.confusion_matrix)

    def _analyze_accuracy(self):
        self.accuracy = np.diag(self.confusion_matrix).sum() / self.confusion_matrix.sum()
        print(f'Accuracy: {self.accuracy}')

    def _analyze_precision(self):
        with np.errstate(divide='ignore', invalid='ignore'):
            try:
                self.precision = np.diag(self.confusion_matrix) / self.confusion_matrix.sum(axis=0)
            except:
                self.precision = np.nan
        print(f'Precision: {self.precision}')

    def _analyze_recall(self):
        with np.errstate(divide='ignore', invalid='ignore'):
            try:
                self.recall = np.diag(self.confusion_matrix) / self.confusion_matrix.sum(axis=1)
            except:
                self.recall = np.nan
        print(f'Recall: {self.recall}')

    def _analyze_f1(self):
        self.f1 = 2 * (self.precision * self.recall) / (self.precision + self.recall)
        print(f'F1: {self.f1}')


In [7]:
analyzer = Analyzer(result["rate"], result["rate_random"], result["rate_random_class"])

analyzer.analyze_error()

MAE: 4.3634211580919136
MSE: 27.162306847404295
RMSE: 5.211747005314465
MAPE: 0.6164334731383864


In [8]:
analyzer.analyze_confusion_matrix()

Confusion Matrix :
[[   0    0    0    0    0    0    0    0    0    0    0]
 [ 177  407  417  404  405  427  452  483  504  494  258]
 [  24   50   40   46   48   57   70   55   69   67   39]
 [  23   31   46   49   46   45   52   49   55   74   48]
 [  30   57   48   60   60   71   84   85   83   92   58]
 [  56  101  112  107  111  103  131  159  172  196  114]
 [  77  167  171  201  195  199  231  267  275  353  196]
 [ 149  298  317  344  395  391  412  474  541  580  317]
 [ 302  620  708  697  716  739  830  930 1008 1092  566]
 [ 477  946  922  989 1145 1140 1206 1235 1347 1388  760]
 [3770 7583 7669 7910 8149 8348 8476 8849 9020 9301 4773]]
Accuracy: 0.07245970686639154
Precision: [0.         0.03966862 0.00382775 0.0045341  0.00532387 0.00894097
 0.01934025 0.03766089 0.07709959 0.10178192 0.66951887]
Recall: [       nan 0.09191509 0.07079646 0.09459459 0.08241758 0.07562408
 0.0990566  0.11237553 0.12280702 0.12012116 0.05692443]
F1: [       nan 0.05541939 0.00726282 0.00865

# Average rate Recommend

In [9]:
df = rates_df.copy()
df.groupby('movie')['rate'].transform('mean')

0         9.270981
1         9.120000
2         9.205059
3         9.317726
4         9.132075
            ...   
140705    8.882759
140706    8.882759
140707    8.882759
140708    8.882759
140709    8.882759
Name: rate, Length: 140710, dtype: float64

In [18]:
class AvgMovieRecommender:
    def __init__(self, movies_df, rates_df):
        self.movies_df = movies_df
        self.rates_df = rates_df

    def run(self, n_items=10):
        df = self.rates_df.copy()
        df['rate_avg_movie'] = df.groupby('movie')['rate'].transform('mean')

        top_n = (
            df.groupby('user', group_keys=False)
            .apply(lambda x: x.nlargest(n_items, 'rate_avg_movie'))
        )

        top_n = top_n.reset_index(drop=True)
        top_n['rate_avg_movie_class'] = top_n['rate_avg_movie'].apply(lambda x: np.round(x))

        return top_n

In [19]:
result = AvgMovieRecommender(movies_df, rates_df).run(20)
result

/var/folders/ph/w9kxghkd5959cwdbzq0m7_9m0000gn/T/ipykernel_52668/3108880182.py:11: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  df.groupby('user', group_keys=False)


,user,movie,rate,time,rate_avg_movie,rate_avg_movie_class
0,0,10200,10,1483582800,9.434136,9.0
1,0,10048,7,1394114760,9.378706,9.0
2,0,10038,9,1495625940,9.343516,9.0
3,0,10021,9,1424497980,9.317726,9.0
4,0,10185,7,1473424740,9.299180,9.0
...,...,...,...,...,...,...
117757,52023,10998,10,1421679660,8.882759,9.0
117758,52024,10998,10,1204622460,8.882759,9.0
117759,52025,10998,7,1498546920,8.882759,9.0
117760,52026,10998,9,1434090420,8.882759,9.0


In [20]:
analyzer = Analyzer(result["rate"], result["rate_avg_movie"], result["rate_avg_movie_class"])
analyzer.analyze_error()

MAE: 1.255901668956898
MSE: 3.952888516210945
RMSE: 1.9881872437501817
MAPE: 0.42152373306057334


In [21]:
analyzer.analyze_confusion_matrix()

Confusion Matrix :
[[    0     1     1    11    68    56   316   406  3570     4]
 [    0     0     0     4     9    13    27   117   321     0]
 [    0     0     0     0    11    13    47    88   283     0]
 [    0     0     0     3     5    14    45   166   424     0]
 [    0     0     1     2     8    25    70   256   846     0]
 [    0     0     0     0    16    27   105   484  1456     5]
 [    0     0     0     2    14    31   146   746  2903     7]
 [    0     0     0     0    14    31   238  1357  6288    17]
 [    0     0     0     1    14    22   151  1416  9957    40]
 [    0     0     0     7    41   114   878  5497 78155   351]]
Accuracy: 0.10061819602248603
Precision: [       nan 0.         0.         0.1        0.04       0.07803468
 0.07217004 0.12883319 0.09555387 0.82783019]
Recall: [0.         0.         0.         0.00456621 0.00662252 0.01290014
 0.03793193 0.17079924 0.85828808 0.00412732]
F1: [       nan        nan        nan 0.00873362 0.01136364 0.02214022
 0.0

/var/folders/ph/w9kxghkd5959cwdbzq0m7_9m0000gn/T/ipykernel_52668/2850479046.py:68: RuntimeWarning: invalid value encountered in divide
  self.f1 = 2 * (self.precision * self.recall) / (self.precision + self.recall)
